In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from betedge_data import BetEdgeClient
from betedge_data import ClientOptionRequest

client = BetEdgeClient()

request = ClientOptionRequest(
    root="AAPL",
    start_date="20240101",
    end_date="20240131",
    data_schema="quote",
    interval=900000,  # 15 minutes in milliseconds
    force_refresh=False,
)

job_info = client.request_data(request)

print(job_info)

JobInfo(job_id=UUID('0206947f-af31-4f2c-9352-02d340e15578'), status=<JobStatus.COMPLETED: 'completed'>, total_items=21, completed_items=21, error_message=None, created_at=datetime.datetime(2025, 9, 26, 20, 15, 47, 954434, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 9, 26, 20, 15, 47, 954434, tzinfo=datetime.timezone.utc))


In [27]:
df = client.retrieve_data(request)

In [37]:
import polars as pl
df.filter((pl.col("bid_size") > 0) & (pl.col("ask_size") > 0)).count()

ms_of_day,bid_size,bid_exchange,bid,bid_condition,ask_size,ask_exchange,ask,ask_condition,date,root,expiration,strike,right
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937,1025937


In [ ]:
df = df.filter(
    (pl.col("bid_size") > 0) & (pl.col("ask_size") > 0)
                ).with_columns([
                    ((pl.col("bid") + pl.col("ask"))/2).alias("mid"),
                    (pl.col("bid") - pl.col("ask")).abs().alias("spread"),
                ])


ms_of_day,bid_size,bid_exchange,bid,bid_condition,ask_size,ask_exchange,ask,ask_condition,date,root,expiration,strike,right,mid,spread
u32,u32,u32,f32,u32,u32,u32,f32,u32,u32,str,u32,u32,str,f32,f32
35100000,34,31,134.649994,50,14,6,138.100006,50,20240102,"""AAPL""",20240119,50000,"""C""",136.375,3.450012
36000000,30,60,134.800003,50,30,60,139.0,50,20240102,"""AAPL""",20240119,50000,"""C""",136.899994,4.199997
36900000,30,73,136.5,50,30,73,136.899994,50,20240102,"""AAPL""",20240119,50000,"""C""",136.699997,0.399994
37800000,33,43,136.100006,50,34,47,136.550003,50,20240102,"""AAPL""",20240119,50000,"""C""",136.325012,0.449997
38700000,10,47,135.5,50,30,47,135.800003,50,20240102,"""AAPL""",20240119,50000,"""C""",135.649994,0.300003
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
54000000,3,1,185.970001,0,3,60,185.990005,0,20240131,"""AAPL""",0,0,"""U""",185.980011,0.020004
54900000,1,60,185.490005,0,1,1,185.5,0,20240131,"""AAPL""",0,0,"""U""",185.494995,0.009995
55800000,8,65,185.509995,0,3,1,185.529999,0,20240131,"""AAPL""",0,0,"""U""",185.519989,0.020004


In [42]:
stock = df.filter(pl.col("expiration") == 0)
joined = df.filter(pl.col("expiration") != 0).join(
    stock, on=["ms_of_day", "date"]
)
joined

ms_of_day,bid_size,bid_exchange,bid,bid_condition,ask_size,ask_exchange,ask,ask_condition,date,root,expiration,strike,right,mid,spread,bid_size_right,bid_exchange_right,bid_right,bid_condition_right,ask_size_right,ask_exchange_right,ask_right,ask_condition_right,root_right,expiration_right,strike_right,right_right,mid_right,spread_right
u32,u32,u32,f32,u32,u32,u32,f32,u32,u32,str,u32,u32,str,f32,f32,u32,u32,f32,u32,u32,u32,f32,u32,str,u32,u32,str,f32,f32
35100000,34,31,134.649994,50,14,6,138.100006,50,20240102,"""AAPL""",20240119,50000,"""C""",136.375,3.450012,1,1,186.25,0,7,60,186.259995,0,"""AAPL""",0,0,"""U""",186.255005,0.009995
36000000,30,60,134.800003,50,30,60,139.0,50,20240102,"""AAPL""",20240119,50000,"""C""",136.899994,4.199997,3,65,186.509995,0,3,60,186.529999,0,"""AAPL""",0,0,"""U""",186.519989,0.020004
36900000,30,73,136.5,50,30,73,136.899994,50,20240102,"""AAPL""",20240119,50000,"""C""",136.699997,0.399994,4,3,186.520004,0,1,1,186.529999,0,"""AAPL""",0,0,"""U""",186.524994,0.009995
37800000,33,43,136.100006,50,34,47,136.550003,50,20240102,"""AAPL""",20240119,50000,"""C""",136.325012,0.449997,2,7,186.179993,0,4,3,186.190002,0,"""AAPL""",0,0,"""U""",186.184998,0.01001
38700000,10,47,135.5,50,30,47,135.800003,50,20240102,"""AAPL""",20240119,50000,"""C""",135.649994,0.300003,4,68,185.520004,0,6,3,185.539993,0,"""AAPL""",0,0,"""U""",185.529999,0.019989
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
54000000,51,7,191.550003,50,50,1,195.75,50,20240131,"""AAPL""",20250321,380000,"""P""",193.649994,4.199997,3,1,185.970001,0,3,60,185.990005,0,"""AAPL""",0,0,"""U""",185.980011,0.020004
54900000,56,1,192.149994,50,50,1,196.800003,50,20240131,"""AAPL""",20250321,380000,"""P""",194.475006,4.650009,1,60,185.490005,0,1,1,185.5,0,"""AAPL""",0,0,"""U""",185.494995,0.009995
55800000,50,1,193.0,50,50,60,196.850006,50,20240131,"""AAPL""",20250321,380000,"""P""",194.925003,3.850006,8,65,185.509995,0,3,1,185.529999,0,"""AAPL""",0,0,"""U""",185.519989,0.020004


In [44]:
from betedge_data import ClientEarningsRequest

earnings_request = ClientEarningsRequest(
    start_date="202401",
    end_date="202510",
)

client = BetEdgeClient()
client.request_data(earnings_request)

2025-09-26 21:07:40 | betedge_data.client.client | INFO | Processing request for force_refresh=False start_date='202401' end_date='202510' return_format='parquet'.
2025-09-26 21:07:40 | betedge_data.alternative.earnings.earnings_client | INFO | Starting monthly earnings data fetch for 2024-01
2025-09-26 21:07:40 | betedge_data.alternative.earnings.earnings_client | INFO | Starting monthly earnings data fetch for 2024-02
2025-09-26 21:07:40 | betedge_data.alternative.earnings.earnings_client | INFO | Generated 21 trading dates for 2024-01
2025-09-26 21:07:40 | betedge_data.alternative.earnings.earnings_client | INFO | Generated 20 trading dates for 2024-02
2025-09-26 21:07:47 | betedge_data.alternative.earnings.earnings_client | INFO | Data collection complete: 2277 records from 20/20 dates
2025-09-26 21:07:47 | betedge_data.alternative.earnings.earnings_client | INFO | Converted 2277 earnings records to Parquet
2025-09-26 21:07:47 | betedge_data.alternative.earnings.earnings_client | I

JobInfo(job_id=UUID('d9ae7cbc-e9aa-40ce-aaee-3e3f9b57b723'), status=<JobStatus.COMPLETED: 'completed'>, total_items=22, completed_items=22, error_message=None, created_at=datetime.datetime(2025, 9, 27, 1, 7, 40, 657231, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 9, 27, 1, 7, 40, 657231, tzinfo=datetime.timezone.utc))

In [57]:
import pandas as pd
import requests
from io import StringIO

def get_index_tickers(index_name):
    urls = {
        'sp500': 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies',
        'dow': 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average',
        'nasdaq100': 'https://en.wikipedia.org/wiki/NASDAQ-100',
        'russell1000': 'https://en.wikipedia.org/wiki/Russell_1000_Index',
        'sp400': 'https://en.wikipedia.org/wiki/List_of_S%26P_400_companies',
        'sp600': 'https://en.wikipedia.org/wiki/List_of_S%26P_600_companies'
    }
    
    if index_name not in urls:
        raise ValueError(f"Index {index_name} not supported")
    
    url = urls[index_name]
    
    # Add headers to avoid 403 error
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # Use requests to get the page content
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes
    
    # Read HTML tables from the response content
    tables = pd.read_html(StringIO(response.text))
    
    # Different pages have different structures
    if index_name == 'sp500':
        return tables[0]['Symbol'].tolist()
    elif index_name == 'dow':
        return tables[1]['Symbol'].tolist()
    elif index_name == 'nasdaq100':
        return tables[4]['Ticker'].tolist()
    else:
        # Try common column names
        for col in ['Symbol', 'Ticker', 'Stock Symbol']:
            if col in tables[0].columns:
                return tables[0][col].tolist()
    
    return []

# Usage examples
try:
    sp500_tickers = get_index_tickers('sp500')
    print(f"S&P 500: {len(sp500_tickers)} tickers")
    print(sp500_tickers[:5])  # First 5 tickers
except Exception as e:
    print(f"Error: {e}")

S&P 500: 503 tickers
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']


In [62]:
sp500_tickers = [x.replace(".","") for x in sp500_tickers]
sp500_tickers


['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APP',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRKB',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'XYZ',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BFB',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'COIN',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 '

In [ ]:
from betedge_data import ClientOptionRequest

for ticker in sp500_tickers:
    req = ClientOptionRequest(
        root=ticker,
        start_date="20200101",
        end_date="20250926",
        data_schema="eod"
    )

    client.request_data(req)

2025-09-26 22:30:31 | betedge_data.client.client | INFO | Processing request for force_refresh=False root='MMM' start_date='20200101' end_date='20250926' data_schema='eod' interval=900000 return_format='parquet' start_time=None end_time=None.
2025-09-26 22:30:31 | betedge_data.historical.option.option_client | INFO | Starting data fetch for root='MMM' data_schema='eod' date=None yearmo=202112 interval=900000 return_format='parquet' exp=0
2025-09-26 22:30:31 | betedge_data.historical.option.option_client | INFO | Starting data fetch for root='MMM' data_schema='eod' date=None yearmo=202209 interval=900000 return_format='parquet' exp=0
2025-09-26 22:30:36 | betedge_data.common.http | INFO | Fetched 1 pages with 1172 total items
2025-09-26 22:30:40 | betedge_data.common.http | ERROR | HTTP error on page 1: 475: View log file for more information: java.lang.ArrayIndexOutOfBoundsException
2025-09-26 22:30:40 | betedge_data.historical.option.option_client | ERROR | Data processing failed for 